In [2]:
import gatenlp
from tqdm import tqdm

import re


def get_mention(text):
    pattern = r"\[START_ENT\](.*?)\[END_ENT\]"
    match = re.search(pattern, text)
    if match:
        return match.group(1).strip()
    return None


def convert_to_gatenlp_doc(data):
    # get mention between [START_ENT] and [END_ENT]

    # Create a GATE document
    doc = gatenlp.Document()
    doc.text = data["input"].replace("[START_ENT]", "").replace("[END_ENT]", "")
    # add "gold" annotation set
    annset = doc.annset("gold")
    ment = get_mention(data["input"])
    # find start and end index
    start = doc.text.find(ment)
    end = start + len(ment)

    # add "mention" annotation with linking features

    ann = annset.add(start=start, end=end, anntype="MISC", features=[])

    return doc


# Your dataset
dataset = {
    "id": 0,
    "input": "[START_ENT] Bandar Seri Begawan [END_ENT] 11 15 AFP ...",
    "output": [
        {
            "answer": "Bandar Seri Begawan",
            "provenance": [{"title": "Bandar Seri Begawan"}],
        }
    ],
    "meta": {
        "left_context": "",
        "right_context": "11 15 AFP The United States today Wednesday deemed ...",
        "mention": "Bandar Seri Begawan",
    },
    "candidates": ["Bandar Seri Begawan", "Brunei International Airport"],
}

# read jsonl file
import json

with open("./Datasets/wnum/wnum.json") as f:
    data = json.load(f)

gatenlp_docs = []
for item in tqdm(data):
    gatenlp_docs.append(convert_to_gatenlp_doc(item))

# Print the GATE document
print(len(gatenlp_docs))
# dump to jsonl file

# with open('ace2004-test-kilt.gatenlp.jsonl', 'w') as outfile:
#     for entry in tqdm(gatenlp_docs):


#         outfile.write(json.dumps(entry.to_dict())+"\n")

100%|██████████| 100002/100002 [00:02<00:00, 36219.84it/s]

100002


In [10]:
!pip3 install gatenlp

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.6/335.6 KB 6.3 MB/s eta 0:00:00a 0:00:01


In [3]:
import requests

api_biencoder = 'http://localhost:20980/api/blink/biencoder/mention/doc'
api_indexer = 'http://localhost:20982/api/indexer/search/doc/10'

from requests.auth import HTTPBasicAuth
auth = HTTPBasicAuth('DS2023', 'eexeegheichai3OhChi5AhcheecaaShe')

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.0.4) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [4]:
# help functions

import numpy as np
import base64

def vector_encode(v):
    s = base64.b64encode(v).decode()
    return s

def vector_decode(s, dtype=np.float32):
    buffer = base64.b64decode(s)
    v = np.frombuffer(buffer, dtype=dtype)
    return v

In [17]:
gatenlp_docs[0].to_dict()

{'annotation_sets': {'gold': {'name': 'gold',
   'annotations': [{'type': 'mention',
     'start': 1,
     'end': 20,
     'id': 0,
     'features': {}}],
   'next_annid': 1}},
 'text': ' Bandar Seri Begawan  11 15 AFP The United States today Wednesday deemed the order issued by Palestinian President Yasser Arafat for a ceasefire in territories under Palestinian Authority control as a positive gesture but considered that it does not release constitute a release form the terms of the Sharm el Sheikh agreement James Stewart the White House s spokesman in Bandar Seri Begawan the capital of the Sultanate of Brunei which American President Bill Clinton is visiting said of course we positively welcome the announcement aimed at stopping the violence But the important point is that Palestinian and Israeli officials take the right',
 'features': {},
 'offset_type': 'p',
 'name': ''}

In [35]:
gdoc_encoding

Document(Bandar Seri Begawan 11 15 AFP The  United States  today Wednesday deemed the order issued by Palestinian President Yasser Arafat for a ceasefire in territories under Palestinian Authority control as a positive gesture but considered that it does not release constitute a release form the terms of the Sharm el Sheikh agreement James Stewart the White House s spokesman in Bandar Seri Begawan the capital of the Sultanate of Brunei which American President Bill Clinton is visiting said of course we positively welcome the announcement aimed at stopping the violence But the important point is that Palestinian and Israeli officials take the right steps to provide a safer climate,features=Features({'pipeline': ['biencoder']}),anns={'gold': AnnotationSet([Annotation(35,48,MISC,features=Features({}),id=0)]), 'entities_merged': AnnotationSet([Annotation(35,48,MISC,features=Features({'linking': {'encoding': '4uKXPeMrxrv5f+w9yVGuPWNsLb4yoW+8K4Ylvh/umL2UZco8Qd8EvdZrIT3KsgM+2goKvq+NlL0yiYK+ekRfPPfcXz1vFmu9WQkMPk/6g7p2Rw6+1woJPi4kVTzDkCO+yLYsvhtgBr0vOkk92f0dPSYPBj6mLjM+CM4Dvc0kEb4L8wY+6X08vhHzKr2m7u+8ayvYPfDif70c/h++uDTXPTSi7L0B+169Cb6HPe0L/LyTGjC+/IufvmGd9L1ECQO+ObdhPs1BXz37m4490sBJPb521ztLIoc98axhvQ8pfb5jPu29w66WvWRRHL0LsJg9KMNwPDsX6D270ac9/wLLvZmZTT2LJ5k729gcPbMz8z37Z2o+FrCrvaFFuj1koxe+vC5svUulSD6sU+e9A5+KPbEDML6JrAk9TCdhvnhnPLvCWcM8eR/hPd5xzrwCu8s7m5MTPG7aEr24wMe9iseXvTy3Vb0+WbK9BiUEvi0w7bs5BOa8U0YzvdGd8DxDBZg7E6J2vMjpizw3pZm9lDAhvhypvr3LoM89G9dBvuDGFT6LMDC9l/W3vEkKxrx5Vb69CR/OPWcxLL43aHm7toPHPN/Lvj1N32o915sHu9vQ47yH5PK8DXB3PD/JVb0J8y8+GYZkPbglHb1Mk2M97ULavJwW+DsjHPO8WRPnvTc5sz2J1PE9yls9uiaPhj2gZ2w9elBIPdD9nb2lFIA9MIjlPf0FsL2MXwY9mOIxPlhRB77c1NU98WURPpmpyLlR96M8O4QDPoh5Oj3+TLK9BGZ5Pf3AJL7ivAe+ybguPpxUpj6Zwxg+OTFSPUFzlTwTpGs+/usjvScrQD0yLPo8d1JVvGdjPDoQ8Dw+EaFkPEjKK76+s6q9+UKavY++EL5tL6g94Qt/vopdIr7L1hA9s+TOvUj+vb7NhYY9Duf6PMhiHrxiZk2+in9TuhM68r27KCS9/kLkveRGGj3BkpO8Wv8CvcYlVLz4SNY81Ro9vjwZ7b3kNw++T0SCvIPbFz5LN9a9we/5vGg/WT0CVBo+NMp2Ph+ZHr76XDK+MQthvS53+T3lWsg8gBujPO4aqL1AtJQ98S0Gvmo8YL54RYE9rRskPulokr2JiYK94IKEPWiK6zz9+Uy9yg+uvYKUI7wAHc89u1UmvnuDrD5anDU88CWHPHAwjT3qcTm94G2tPQN03DwtuqK8AX/LvXQWHL3p/+o99mBBPPceG72d/x++LH+EO0tGAL46mhg+RzQRPq8+yD3yAik9EFHou/IzqTzR0yu9ONTgvCgTk73FHQY+9R75vFZgIj4BO6s9NfplvaWT7r083lU9/lH8u0FeP70vhJI9ry/hPeiVAD4YKR29JXOwPDsKUj6RkJg9oYuTvWycmr40MXa+sRzgPRtpujw3T629s5AKvjee/T06+Wa8O4ZBPTY5ZL5r7y25ZJ0wveqe4b3iYNM7oR3AvNGQBL64D2S9Q6MXPZGrjzzsN7Y9FKfIvD6O6Tw8SAS9olvAvQp4xT3sWv29l+08ve3xzDz8ArE9YaK1Pa6PHjs7Mfo8z/WcPbCbMj253+I9mO9uOzE1+ryNdIo7oAaEPYYQI7w++SG+cHKCPWw3u77JA9a9kI3svGeamb1bUyE+x8Q+Pl3Ymj22Fig73k5kvTmF7rz5Vi29xlFTPSnwxr3qpVk983GLPKtxCz171Sg9bH/Hvap5QD37eKe99zvXPV2clL3DiJi9guKqvWNhzL3eRBw9vfYEvu3OtT36fOe9DCQzvfosobzrFpY9u+0gvnHRI75AiuK9ROPBPUDfGL7pYqe7uP7DPfyjET5cbji+VZfivf3MT75ezWe9WbIEPY821rsog+m9/EisveNH+L1/rzQ9S7k3vmSV4jtIFHY7G8sAvs8G8T35woE7WkHYOQBW2b0/ik0+rWmePZUxA746Tga+/0puPb/mzr3TuJe+LxsoPRZUqrz+hZQ6+Fxkvn9ryT3RwuK9Szn7vaE+cr7WVbG95DgFPCD7/zzRNC88g1KKPTryZr1OMJg8h5CBvs0zFz6PlvM8GHPDPLtsCz5iVUW+HXmJvSb/arw5kc69fdsZPlGJVL6ZbDE+dUwMvhyGYD1xkrC9OIX1PTl3HL6uuyo+cqwTvWE8Zz17k4a9IIjoPZnTeL2sknq7pUKePOx5LL2/4mk9sDvWPT9/Hb5ydPm7YEuAvR1LU70W8VS9A7QlviOioL3e3ok9jkflu+JbJr2d4AQ9zxgPviNoJr0r9MK9M657PS4aTLv4pwi+f5w1vr7z2L0bB/k90r3TPKj2tL3f0wE9GupivUs7zb1C8xa+e72MvG5msT3Hip+9JXsgPCOHPr5q6pa8GZCdPbGR4D3XhF+9FD9dvtOY3r2iZa89DpW8PPeaQz1yPQg9+TdIPpvyNL0Jw748WnKEvHcbK77qPAU+EppbPgKFBb1Luw4+sZq3O8gZmz0Mq127vKiPPWGFSb4Wsg++myDhPeRHmrwEIki9gKD2PWUzCT5OdWu9YSe4PXUx7L3kQdA9PJazvVXK8L3nu5u9VNzNPa9ddTqdmrc8OR4Ivh9qnrsjWiW+p49yvqi3Uj10we+9vKiIPRmssLwZrvy9x4QUPm0NJ71Rr4Q8WiWAPV825r2s92q+OiGHvXysKr4zLfm9PQDRPOpnMT6XuSE+2rYVPfasur0bmDA+x2cAvSfgRbxzI6a93TRgvuo0wL2Rg+w7cLnlPCIcDr3G84u9qRliPJTjqjsbSeQ8oT7FPUGW/byN0d68HxNIPRJe9D1I5cq9vNmCOykvK76rGQi+TuETPkv6tb0WlpW9vOONvaDfhT1rLdI8kJ+FvqDiGr7Neo09qce4vR/s871hBOQ9+GSIOuB9ZD6TMRU+Q5mqPWaByT0K37i9FcWMPXxqFbmpJuC9H9AovtSEX7zGOK64zc9+vQimiL3/HYu9Id/oPPN8GL48w5m8vP5pPcT7MT1ec7c88LV5PTewe76xp6q9km7YPWSx/rwbzqw8ONAsvTvACz0CEac9iozMPE4qx7xpRAW+CPAUvsaE/D3WPxo++/+hvT0nA74y0j08lPSjvVKNNT

In [5]:
from gatenlp import Document

linked_docs = []


for item in tqdm(gatenlp_docs):
  try:
    # rename needed to use the API
    gold_dict = item.to_dict()
    gold_dict['annotation_sets']['entities_merged'] = gold_dict['annotation_sets']['gold']
    gold_dict['annotation_sets']['entities_merged']['name'] = 'entities_merged'
    for ann in gold_dict['annotation_sets']['entities_merged']['annotations']:
      ann['features'] = {}
    gold_annotations_doc = Document.from_dict(gold_dict)
    gold_annotations_doc.features['pipeline'] = []
    res = requests.post(api_biencoder, auth = auth, json = gold_annotations_doc.to_dict())
    doc = res.json()
    gdoc_encoding = Document.from_dict(doc)
    res = requests.post(api_indexer, auth = auth, json = gdoc_encoding.to_dict())
    doc = res.json()
    gdoc = Document.from_dict(doc)
    linked_docs.append(gdoc)
  except Exception as e:
    print(e)
  

100%|██████████| 100002/100002 [3:16:27<00:00,  8.48it/s]  


In [6]:
index = 0
nils = 0
for item in tqdm(linked_docs):
    candidates = item.to_dict()["annotation_sets"]["entities_merged"]["annotations"][0][
        "features"
    ]["linking"]["candidates"]
    candidatesList = [candidate["title"] for candidate in candidates]
    data[index]["candidates"] = candidatesList

    if data[index]["output"][0]["answer"] not in candidatesList:
        data[index]["output"][0]["answer"] = "Not In Candidates"
        nils += 1
    index += 1

100%|██████████| 100002/100002 [00:02<00:00, 41115.00it/s]


In [7]:
nils

8234

In [9]:
#dump to jsonl file updated data 
with open('./wnum.jsonl', 'w') as outfile:
    for entry in tqdm(data):
        outfile.write(json.dumps(entry)+"\n")

100%|██████████| 100002/100002 [00:00<00:00, 125625.31it/s]


In [41]:
gdoc.to_dict()['annotation_sets']['entities_merged']['annotations'][0]['features']['linking']['candidates']

[{'raw_score': 325.1113586425781,
  'id': 783851,
  'wikipedia_id': 3434750,
  'wikidata_qid': None,
  'redirects_to': None,
  'title': 'United States',
  'url': 'https://en.wikipedia.org/wiki?curid=3434750',
  'type_': None,
  'indexer': 0,
  'score': 81.96341705322266,
  'norm_score': 0.5423622485900884},
 {'raw_score': 334.17315673828125,
  'id': 15763,
  'wikipedia_id': 31769,
  'wikidata_qid': None,
  'redirects_to': None,
  'title': 'United Nations',
  'url': 'https://en.wikipedia.org/wiki?curid=31769',
  'type_': None,
  'indexer': 0,
  'score': 77.43250274658203,
  'norm_score': 0.46290169663524444},
 {'raw_score': 334.7706298828125,
  'id': 3155783,
  'wikipedia_id': 27552742,
  'wikidata_qid': None,
  'redirects_to': None,
  'title': 'Law of the United States',
  'url': 'https://en.wikipedia.org/wiki?curid=27552742',
  'type_': None,
  'indexer': 0,
  'score': 77.1337661743164,
  'norm_score': 0.5066513418138748},
 {'raw_score': 334.77203369140625,
  'id': 197788,
  'wikipedi